In [1]:
#import all necessary libraries
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
import datetime as dt
from pandas_profiling import ProfileReport
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, RobustScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import KMeans
import matplotlib.cm as cm
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest

In [2]:
######################################################################### download
# load the csv file and save it into the dataframe donors
donors = pd.read_csv(os.path.join(r"C:\Users\betty\OneDrive\Documentos\GitHub\Project\donors.csv"), index_col = 0)
#donors.head()

C:\Users\betty\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# check id unique
#donors['CONTROLN'].unique().size #95412
donors.set_index('CONTROLN',drop=True,inplace=True)

In [4]:
#donors.info(verbose=True,null_counts=True)

In [5]:
donors_original=donors.copy()

In [6]:
#checking for duplicated observations
#donors.duplicated().sum()

**Drop Columns**

In [7]:
# Pandas profiling
#ProfileReport(donors, minimal=True)

In [8]:
#Based on previous analysis, we will drop the following columns (justification provided in report appendix??)

drop_columns =  (['OSOURCE','PVASTATE','NOEXCH','HOMEOWNR','MALEMILI','MALEVET','VIETVETS','WWIIVETS','LOCALGOV','STATEGOV','FEDGOV','GEOCODE','LIFESRC','HPHONE_D','GEOCODE2'] +
                #drop all ADATE columns except last promotion 17NK        
                list(donors.filter(regex='^ADATE_',axis=1).columns.values)[1:] +
                #drop all the RFA columns
                list(donors.filter(regex='^RFA_',axis=1).columns.values)[:-3] +
                #drop all the RDATE columns
                list(donors.filter(regex='^RDATE_',axis=1).columns.values) + 
                #drop all the RAMNT columns
                list(donors.filter(regex='^RAMNT_',axis=1).columns.values))
                
donors.drop(drop_columns, axis=1,inplace=True)

In [9]:
#treat column MAJOR to see if the information matches the one in the column  MDMAUD, regarding no major donors, and if yes delete MAJOR

#check if all the donors with blank cells (not a major donor) under MAJOR correspond a donors with X value (meaning not a major donor) under  MDMAUD
donors.loc[donors["MAJOR"] ==" "][["MDMAUD"]] ["MDMAUD"].value_counts() #We can conclude that all blank values under column Major (that represent the not major donor) have a correspondent X in the column MDMAUD, so the column Major can be deleted 

#drop column MAJOR
donors.drop("MAJOR",axis=1, inplace=True)

# Also drop MDMAUD since the information is already separated in RFA
donors.drop("MDMAUD",axis=1, inplace=True)

In [10]:
#clean up the TCODE column
#typecast it into a string for treatment
donors['TCODE']=donors['TCODE'].astype(str)
donors['TCODE'].dtypes

dtype('O')

In [11]:
#treat T-code column to only keep the first 3 letters

donors['TCODE']=donors['TCODE'].map(lambda x: x[:-3] if '002' in x or '004' in x or '028' in x else x) # same code - same meaning without the gender and plural

In [12]:
# treat high cardinality choosing the most frequent states and group others as 'other'
#donors['STATE'].value_counts()
state8 = donors['STATE'].value_counts()[:8].index.tolist()
donors['STATE'] = np.where(~donors['STATE'].isin(state8), 'Other', donors['STATE'])

#drop ZIP - redundant
donors.drop('ZIP', axis=1, inplace=True)

In [13]:
# percentage of data with a Bad address
#(len(donors_original[donors_original['MAILCODE']=='B'])/len(donors_original))*100 #1.466% 1399 donors

#since it's such a small percentage of the dataset and given the time & resource constraint for the project
#we decided to drop them
filter_to_drop = donors_original['MAILCODE']=='B'

In [14]:
#drop the rows or donors which have a Bad address 
donors = donors[donors['MAILCODE']!='B']

#now we can drop the mailcode column because it no longer adds information to the data
donors.drop(['MAILCODE'], axis=1,inplace=True)

In [15]:
#check the type of the DOB column
#donors['DOB'].dtypes #it's a string
#treat the DOB to only show the year without the day or month
donors['DOB'] = donors['DOB'].str[:4]

#donors['DOB'].head()

In [16]:
# create AGE 
donors['AGE'] = 2020 - donors['DOB'].astype('float')
donors.drop('DOB',axis=1, inplace=True)

In [17]:
# do same thing for original data
donors_original['DOB'] = donors_original['DOB'].str[:4]
donors_original['AGE'] = 2020 - donors_original['DOB'].astype('float')

In [18]:
# probably parents donating with children name - can bias the clustering 
#donors[(donors['AGE']<16)]
donors = donors[~(donors['AGE']<16)]

In [19]:
filter_to_drop = filter_to_drop | (donors_original['AGE']<16)
#len(donors_original[filter_to_drop])/len(donors_original) *100 #1.90

In [20]:
#len(donors_original[(donors_original['AGE']<16)])#/len(donors_original) *100 # we only have 425 donors younger than 16 years old

In [21]:
# Treat file flags columns with blanks and Xs to 0s and 1s
replaceFlag_dict = {' ': 0,'X': 1}
# Filtering the columns that start with 'REC' - record columns
rec_columns = donors.filter(regex='^REC',axis=1).columns.values
for rec in rec_columns:
    donors[rec].replace(replaceFlag_dict, inplace= True)

In [22]:
#check that it now only includes values 0 and 1
#for rec in rec_columns : 
#    print(donors[rec].value_counts())

In [23]:
#filling missing values of the gender column
#trying to associate it with tcode
donors['TCODE_original'] =donors_original['TCODE'].astype(str)
donors['TCODE_original'].unique()

array(['0', '1', '2', '28', '3', '1002', '42', '4', '18', '980', '14',
       '28028', '72', '22', '13002', '23', '45', '24', '4002', '30', '13',
       '202', '136', '72002', '96', '116', '100', '4004', '39002', '61',
       '47', '36', '228', '14002', '6', '6400', '40', '25', '21', '12',
       '58002', '134', '18002', '38', '9', '76', '50', '27', '93', '17',
       '94', '7', '44', '24002', '22002'], dtype=object)

In [24]:
female_list = ['2','3','28028','42']
male_list = ['1','36','40']
donors['gender_code'] = donors['TCODE_original'].map(lambda x: 'F' if '002' in x or x in female_list else 'M' if x in male_list else ' ')
#donors['gender_code']

In [25]:
# check association with the categories F and M 
GENDER_vs_gender= donors[(donors.GENDER.isin(['F','M']))&(donors.gender_code!=' ')][['TCODE_original','GENDER','gender_code']]
#adding a binary column with 1 if there's no match 
GENDER_vs_gender['inequality'] = np.where(GENDER_vs_gender['GENDER'] == GENDER_vs_gender['gender_code'], 0, 1)

#different gender association
#GENDER_vs_gender[GENDER_vs_gender['inequality']==1]
GENDER_vs_gender[GENDER_vs_gender['inequality']==1]['TCODE_original'].value_counts()
# there a lot of errors associating the gender with tcode (misters has females and misses as males, for example) 
# since we dont know the origin of the error we are not going to do this association given the assumptions that we need to do

2        3320
1002     1489
1         241
4002       34
3          32
39002       6
13002       4
28028       2
42          1
18002       1
24002       1
58002       1
22002       1
Name: TCODE_original, dtype: int64

In [26]:
# drop irrelevant columns (used just to do this association)
# AND tcode has it is lacking information and may not be very useful
donors.drop(['TCODE','TCODE_original','gender_code'], axis=1, inplace= True)

In [27]:
# we decided to replace the missing values with category U - unknown
# replace blanks, C, and A in gender column with U as they are all unknown values
replaceGender_dict = dict.fromkeys([' ', 'C', 'A','J'], 'U')
#print(replaceGender_dict)
donors['GENDER'].replace(replaceGender_dict,inplace= True)

#check that it now only includes values that are listed/categorized in the meta data
#donors['GENDER'].value_counts()

In [28]:
#treat the PEPSTRFL column to have 0s and 1s instead of blanks and Xs
donors['PEPSTRFL'].replace(replaceFlag_dict, inplace= True)
#check that it now only includes values 0 and 1
#donors['PEPSTRFL'].value_counts()

In [29]:
# Associating NUMCHILD WITH 'CHILD03', 'CHILD07', 'CHILD12', 'CHILD18'
childAge_columns = donors.filter(regex='^CHILD', axis=1).columns.values

In [30]:
# we noticed that not all the blanks of NUMCHILD are 0 children. Since we have some of CHILD_ variables filled 
# so for this blanks we are going to put the least number of children this donor has

# If we have this values on CHILD__ we have at least __ child
sum_dict = {"B":2,"M":1,"F": 1," ":0}

# replacement
for age in childAge_columns:
    donors[age].replace(sum_dict, inplace=True)
    
# create a new variable named number_child that will sum the values per row under CHILD03,CHILD07, CHILD12, CHILD18 columns 
donors['number_child']= donors[childAge_columns].sum(axis=1)

# replace blanks by 0's
donors['NUMCHLD'].replace(np.nan,0.0,inplace=True)

donors['NEW_NUMCHLD'] = np.where(donors['number_child']>donors['NUMCHLD'],donors['number_child'],donors['NUMCHLD'])
#donors['NEW_NUMCHLD']

In [31]:
# percentage of values that remained the same when comparing the variable NUMCHLD and the one created
compare_numchld_number_child = np.where(donors["NUMCHLD"] == donors["NEW_NUMCHLD"], True, False)
#print(np.sum(compare_numchld_number_child *1)/len(donors)) # the percentage is 97%

# shows the difference between values under the columns NEW_NUMCHLD and NUMCHLD
#diff = donors['NEW_NUMCHLD'] - donors['NUMCHLD']
#print(diff.value_counts()) #92040 rows are equal between the 2 columns

In [32]:
# stay with new column, delete CHILD__ columns and auxiliar ones
donors['NUMCHLD'] = donors['NEW_NUMCHLD']
donors.drop(['CHILD03','CHILD07','CHILD12','CHILD18','NEW_NUMCHLD','number_child'],axis=1,inplace=True)

In [33]:
# treat column HIT in order to analyse the presence of outliers

#create a boxplot for the variable HIT
#plt.figure(figsize=(30,15))
#sns.boxplot(y=donors["HIT"])
#plt.show()

#create a histogram for the variable HIT
#sns.set() 
#plt.figure(figsize=(30,15))
#sns.countplot(x=donors["HIT"])
#plt.title("Number of times donors replied to mail order offers other than PVA")
#plt.show() #The hits of 32 to 241 account for a very low number of donors


#  detect outliers manually by limiting the number of replies to 31
filters1 = ((donors["HIT"]<=31))
my_project_noutliers = donors[filters1]

# how much data is kept if we limit the number of replies to 31
#print('Percentage of data kept after removing outliers:', np.round(my_project_noutliers.shape[0] / donors.shape[0], 4))

#detect outliers manually by limiting the number of replies to 84
#filters2 = ((donors["HIT"]<=84))
#my_project_f2 = donors[filters2]

#how much data is kept if we limit the number of replies to 84
#print('Percentage of data kept after removing outliers:', np.round(my_project_f2.shape[0] / donors.shape[0], 4))

# decide on keeping only the donors who reply until 31 times
donors= donors[filters1]

In [34]:
filter_to_drop = filter_to_drop | (donors_original['HIT']>31)

In [35]:
#len(donors_original[filter_to_drop]) # 2398
#len(donors_original[filter_to_drop])/len(donors_original) *100 #2.9378

In [36]:
# treat the column SOLP3 so it is possible to integrate the information in the column RECP3 and then delete it, this will happen by distinguishing between donors that are not in P3 program and the ones who can receive always emails

#number of elements under column SOLP3 have a correspondent blank value under RECP3, meaning they represent donors who are not in P3 program.
donors[["RECP3","SOLP3"]].groupby(by=["RECP3"]).count()

#number of blank cells under SOLP3 which can account for missing values,donors who can be mailed as many times as we want or not member of P3 program. .
donors["SOLP3"].isin([" "]).sum(axis=0)

#index of the donors who don't belong to P3 program because they have a blank value under RECP3
grouped= donors[["RECP3","SOLP3"]].groupby(by=["RECP3"]).get_group(0).index

#transform the values under the column SOLP3  correspondent to the above axis in -1
solp31=donors.loc[grouped]["SOLP3"].replace(" ",-1)

#index of the donors who  belong to P3 program because they have a X under RECP3
grouped1=donors[["RECP3","SOLP3"]].groupby(by=["RECP3"]).get_group(1).index

#transform the donors under SOLP3 correspondent to the above axis and assume that they can be mailed as many times as we want to the number 12
solp32=donors.loc[grouped1]["SOLP3"].replace(" ",12)

#return the column SOLP3 with the necessary changes
donors["SOLP3"]=pd.concat([solp31,solp32])

#confirm the values under SOLP3
#donors["SOLP3"].unique()

#transform the strings in integer
donors["SOLP3"].replace({"SOLP3": {"00":0,"12":12,"01": 1,"02":2}}, inplace=True)

#confirm the values under SOLP3
#donors["SOLP3"].unique()

#drop the column RECP3
donors.drop(["RECP3"], axis=1,inplace=True)

In [37]:
#treat column SOLIH so it is possible to integrate the information in the column RECINHSE and then delete it, this will happen by distinguishing between donors that are not in IN House program and the ones who can receive always emails

#number of elements under column SOLIH have a correspondent blank value under RECINHSE, meaning they represent donors who are not inHouse program.
donors[["RECINHSE","SOLIH"]].groupby(by=["RECINHSE"]).count()

#index of the donors who don't belong to In House  program because they have a blank value under RECINHSE
SOLIHg_index1= donors.loc[donors['RECINHSE']==0].index.values

#transform the donors who don't belong to In House program under SOLIH in -1.
SOLIH1=donors.loc[SOLIHg_index1]["SOLIH"].replace(" ",-1)

#index of the donors who  belong to In House program because they have a X under RECINHSE
SOLIHg_index2=donors.loc[donors['RECINHSE']==1].index.values

#transform the donors under SOLIH" correspondent to the above axis and assume that they can be mailed as many times as we want to the number 12
SOLIH2=donors.loc[SOLIHg_index2]["SOLIH"].replace(" ",12)

#return the column SOLIH with the necessary changes
donors["SOLIH"]=pd.concat([SOLIH1,SOLIH2])

#confirm the values under SOLIH
#donors["SOLIH"].unique()

#transform the strings in integer
donors.replace({"SOLIH": {"00":0,"12":12,"01":1,"02":2,"03":3,"04":4,"06":6}}, inplace=True)

#confirm the values under SOLIH
#donors["SOLIH"].unique()

#drop the column RECINHSE
donors.drop(["RECINHSE"], axis=1,inplace=True)

In [38]:
# WEALTH1 vs INCOME
#print('WEALTH1 -', donors['WEALTH1'].isna().sum()/len(donors)*100)
#print('INCOME -', donors['INCOME'].isna().sum()/len(donors)*100)
# All of the INCOME missing values are in common with WEALTH1 !
#donors[donors['INCOME'].isna()]['WEALTH1'].isna().sum()/donors['INCOME'].isna().sum()

In [39]:
#donors[['WEALTH1','WEALTH2']].corr(method='spearman') # highly correlated

In [40]:
#donors['WEALTH2'].isna().sum()/len(donors) *100
# Drop WEALTH1 since it has more missing values and its very correlated with WEALTH2
donors.drop('WEALTH1', axis=1, inplace=True)
# Use WEALTH2 for knn imputation and then drop 

In [41]:
# The following variables indicate the number of known times the donor has responded to other types of mail order offers. 
# Convert them into Y/N columns like the interests variables because anyways there are too many missing values in these HIT variables 
# (with this solution we will lose the degree of interest but its only for 50% of the data which is not a problem)
mail_offers = list(donors.filter(regex='^MB',axis=1).columns.values) + list(donors.filter(regex='^MAG',axis=1).columns.values)+ list(donors.filter(regex='^PUB',axis=1).columns.values)
#donors[mail_offers]

In [42]:
for mail in mail_offers: donors[mail] = donors[mail].apply(lambda x: 1 if x>0 else 0) 

In [43]:
# Convert Y/' ' of the interests variables into 1/0
replaceBool_dict = {'Y': 1,' ': 0}
interests = ['COLLECT1', 'VETERANS', 'BIBLE', 'CATLG', 'HOMEE',
       'PETS', 'CDPLAY', 'STEREO', 'PCOWNERS', 'PHOTO', 'CRAFTS',
       'FISHER', 'GARDENIN', 'BOATS', 'WALKER', 'KIDSTUFF', 'CARDS',
       'PLATES']
for interest in interests: donors[interest].replace(replaceBool_dict, inplace = True)

In [44]:
# Treating redundant variables regarding the donor interests and the response to an email about that interest
##common_topics - crafts, gardening, collectables, photos, family stuff 
common_topics =[['MBCRAFT', 'CRAFTS'],['MBGARDEN','GARDENIN'],['PUBGARDN','GARDENIN'],['MBCOLECT','COLLECT1'],['MBCOLECT','PLATES'],['PUBPHOTO', 'PHOTO'],['MAGFAML','KIDSTUFF']]

In [45]:
# if the donor responds to an email = he has interest
for topic in common_topics:
        donors[topic[1]] = np.where(donors[topic[0]]==1, 1, donors[topic[1]])

In [46]:
# drop columns
donors.drop(mail_offers, axis=1, inplace=True)

In [47]:
##### Treating dates from promotion and gifts history
dates= ['ADATE_2','FISTDATE','NEXTDATE','MAXADATE','MINRDATE','MAXRDATE','LASTDATE']
for d in dates: donors[d] = pd.to_datetime(donors[d])

In [48]:
# Treat missing values
#NA = donors[(donors['FISTDATE'].isna()) | (donors['NEXTDATE'].isna()) | (donors['TIMELAG'].isna())]
#NA[['FISTDATE','NEXTDATE','TIMELAG']].isna().sum()

In [49]:
without_1stdate = donors.iloc[np.where(donors['FISTDATE'].isna())[0]]
#without_1stdate

In [50]:
# Really high timelags
#for i in without_1stdate.index.values:
#    print(relativedelta(months= without_1stdate.loc[i]['TIMELAG']))
#    print(without_1stdate.loc[i]['NEXTDATE'] - relativedelta(months= without_1stdate.loc[i]['TIMELAG']))
#    print(donors.loc[i][['NEXTDATE','TIMELAG']])

In [51]:
# create a boxplot
#plt.figure(figsize=(5,15))
#sns.boxplot(y=donors["TIMELAG"])
#plt.show()

# create a histogram
#sns.set() 
#plt.figure(figsize=(30,15))
#sns.countplot(x=donors["TIMELAG"])
#plt.show()
#donors[donors["TIMELAG"]>30]['TIMELAG'].value_counts() # choose min value with 2 donors - 51

In [52]:
filter_highLag =(~(donors['TIMELAG']>51))

In [53]:
filter_to_drop = filter_to_drop | (donors_original['TIMELAG']>51)

In [54]:
#len(donors_original[(donors_original['TIMELAG']>51)])#/len(donors_original) * 100 # 22 donors

In [55]:
#len(donors_original[filter_to_drop])/len(donors_original) * 100 # 2.9598

In [56]:
donors=donors[filter_highLag]

In [57]:
without_2nddate = donors.iloc[np.where(donors['NEXTDATE'].isna())[0]]
#without_2nddate

In [58]:
#without_2nddate['TIMELAG'].isna().sum() # NEXTDATE and TIMELAG with nans in common  - 9790

In [59]:
# Maybe because this donors only have donated once ? 

# check if firstdate = lastdate 
#(np.where(without_2nddate['FISTDATE']==without_2nddate['LASTDATE'],True,False)*1).sum() # all have the same date
#(np.where(donors['FISTDATE']==donors['LASTDATE'],True,False)*1).sum() #9842

In [60]:
OneTime_donors = donors.iloc[np.where(donors['FISTDATE']==donors['LASTDATE'])][['FISTDATE','NEXTDATE','TIMELAG','LASTDATE']]
#OneTime_donors[OneTime_donors['TIMELAG']==0.0]

In [61]:
#donors.iloc[np.where(donors['FISTDATE']==donors['LASTDATE'])]['TIMELAG'].value_counts(dropna=False) # nan - 9790, 0 - 52

In [62]:
# create binary variable to define one time donors 
# replace TIMELAG by 52 (max value) for one time donors
donors['1TIME_DONOR'] = np.where(donors['FISTDATE']==donors['LASTDATE'], 1, 0)
donors['TIMELAG'] = np.where(donors['FISTDATE']==donors['LASTDATE'], 52, donors['TIMELAG'])

In [63]:
#donors[['FISTDATE','NEXTDATE','TIMELAG']].isna().sum() # we re not going to use nextdate so we dont need to treat the missing values

In [64]:
# Present date 
present_date = dt.datetime.strptime('2020-09-01', '%Y-%m-%d')
# Function to calculate the interval of time between meaningful dates (in months, as TIMELAG)
def diff_month(d1,d2):
    return d1.dt.month - d2.dt.month + 12*(d1.dt.year - d2.dt.year)

In [65]:
# Interval between first gift and present date to understand how long the donors started to donate (this one in years)
donors['TENURE'] = 2020 - donors['FISTDATE'].dt.year
#donors['TENURE'].value_counts()

In [66]:
# drop FISTDATE since it's not as informative after the feature engineering of the tenure column
donors.drop(dates, axis=1, inplace= True)

In [67]:
#drop the column ODATEDW as it gives information of the year when donor gave the first gift which corresponds to FISTDATE and we have created variable tenure to tell for how long the donors hare in PVA's database 
donors.drop(["ODATEDW"], axis=1, inplace=True)

In [68]:
# fill missing values of Data Source of Overlay Data
donors['DATASRCE'].replace(' ', 0, inplace=True)
donors['DATASRCE'] = donors['DATASRCE'].astype(int)

In [69]:
#### RFA (recency/frequency/monetary) fields analysis
RFA = ['MDMAUD_R','MDMAUD_F','MDMAUD_A','RFA_2R','RFA_2F','RFA_2A']

#for column in RFA:
#    print(donors[column].value_counts())

# all lapsed donors for RFA_2 - 17NK promotion => recency is irrelevant
donors.drop('RFA_2R', axis=1, inplace=True)

In [70]:
# Ordinal features with strings to numerical 
donors['RFA_2A'].replace({"D":0, "E":1, "F":2, "G":3}, inplace= True)
donors['MDMAUD_F'].replace({"X":0}, inplace= True)
donors['MDMAUD_F'].astype('int64', copy=False)
donors['MDMAUD_A'].replace({'X':0, 'C':1, 'M':2, 'L':3, 'T':4}, inplace= True)

In [71]:
# treat the column DOMAIN to separate the information inside each cell in two columns
urbanicity_level_neighbourhood = [] 
socio_econ_neighbourhood= [] 
 
for i in donors["DOMAIN"]: 
    urbanicity_level_neighbourhood.append(i[0]) 
    if i==' ':
        socio_econ_neighbourhood.append(i[0]) 
    else:
        socio_econ_neighbourhood.append(i[1]) 

donors["urbanicity_level_neighbourhood"]=pd.DataFrame(urbanicity_level_neighbourhood, index =donors.index)
donors["socio_econ_neighbourhood"]=pd.DataFrame(socio_econ_neighbourhood, index=donors.index)

In [72]:
#replace the blank cells with nan
donors['socio_econ_neighbourhood'].replace({" ":np.nan}, inplace = True)
donors['urbanicity_level_neighbourhood'].replace({" ":np.nan}, inplace = True)

In [73]:
donors['urbanicity_level_neighbourhood'].replace({"U":0, "C":1, "S":2, "T":3,"R": 4}, inplace= True)

In [74]:
donors['socio_econ_neighbourhood']=donors['socio_econ_neighbourhood'].astype('float')
donors['urbanicity_level_neighbourhood']=donors['urbanicity_level_neighbourhood'].astype('float')

In [75]:
# lets stay with the 2 bytes separated and drop domain 
donors.drop(['DOMAIN'], axis=1, inplace=True)

In [76]:
#REDUNDANCY

#check the variables that are highly correlated (more than or equal to 90%) and drop one of them to avoid redundancy
#corr_matrix = donors.corr().abs()
#high_corr_var=np.where(corr_matrix>=0.9)
#high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
#high_corr_var

In [77]:
#can drop POP902 (no of families) and POP903 (no of households) since redundant with POP901(no. of persons)
donors.drop(['POP902','POP903'], axis=1,inplace=True)

#can drop ETHC4(%black < Age 15) & ETHC5(%black 15 - 59) since redundant with ETH2(%black)...logically, age of the 
#people in the neighbourhood will not affect the willingness of the donor to donate making this info irrelevant
donors.drop(['ETHC4','ETHC5','ETHC6'], axis=1,inplace=True)

#can drop since the ages of the races are irrelevant and we already have the info on how many white people live in the 
#neighbourhood in ETH1
donors.drop(['ETHC1','ETHC2','ETHC3'], axis=1,inplace=True)

#can drop ETH13(%mexican) & LSC2 (%spanish speaking) since redundant with ETH5(%hispanic)...hispanic includes all 
#latinos (aka spanish speaking)
#drop all language variables due to lack of relevancy
donors.drop(['ETH13','ETH14','ETH15','ETH16','LSC1','LSC2','LSC3','LSC4'], axis=1,inplace=True)

#can drop these Ages because, AGEC variables are more informative and disaggregated...will keep summary variables 
#AGE904 (avg age of population) and AGE907 (%population under 18)
donors.drop(['AGE901','AGE902','AGE903','AGE905','AGE906'], axis=1,inplace=True)

#can drop this children ages because they are not as relevant since don't impact decision to donate...thus we will keep
#the CHIL variable instead since it is more aggregated
child_list=donors.filter(regex='^CHILC',axis=1).columns.values
donors.drop(child_list,axis=1,inplace=True)

#can drop HHD1(%Households w/ Related Children) since redundant with AGE907(%Population Under Age 18) 
donors.drop('HHD1', axis=1,inplace=True)

#can drop all MARR variables since once again it refers to neighbourhood not the donor himself, meaning that if my
#neighbour is widowed, this would not affect my decision to doate to veterans
donors.drop(['MARR1','MARR2','MARR3','MARR4'], axis=1,inplace=True)

#can drop HHAGE variables due to lack of relevancy since the ages of people in a household in a donor's neighbourhood
#won't impact his/her decision to donate
donors.drop(['HHAGE1','HHAGE2','HHAGE3'], axis=1,inplace=True)

#can drop HHD variables because describes who lives in each household while RHP (rooms), HHP (avg & median) & HHN describes number of people per household
#these variables refer to the neighbourhood meaning they have minimal influence on donors' decision to donate...
#so we can drop them due to lack of relevancy
HHD_list=donors.filter(regex='^HHD',axis=1).columns.values
HHN_list=donors.filter(regex='^HHN',axis=1).columns.values
donors.drop(np.concatenate([HHD_list,HHN_list]), axis=1,inplace=True)
donors.drop(['HHP1','HHP2'], axis=1,inplace=True)
donors.drop(['RHP1','RHP2','RHP3','RHP4'], axis=1,inplace=True)

#can drop HHAS variables which demonstrate a 'poor' neighbourhood which is also represented in IC6 (very low income)
donors.drop(['HHAS1','HHAS2','HHAS3','HHAS4'], axis=1,inplace=True)

#since DW variables refer to the style of house unit structures within neighbourhoods, this wouldn't influence donor behaviour
#thus drop them due to lack of relevancy
DW_list=donors.filter(regex='^DW',axis=1).columns.values
donors.drop(DW_list, axis=1,inplace=True)

#since HUPA refers to home style in neighbourhood,this wouldn't influence donor behaviour thus drop them due to lack of relevancy
HUPA_list=donors.filter(regex='^HUPA',axis=1).columns.values
donors.drop(HUPA_list, axis=1,inplace=True)

#can drop all HV variables since it represents average and median value which is a monetary value not a percentage. This
#is a problem because the census was created in 2010 which means the money will not have the same value today...HVP variables 
#are a better substitute
donors.drop(['HV1','HV2','HV3','HV4'], axis=1,inplace=True)

#can drop all RP variables because HVP variables tell us more about the donor
donors.drop(['RP1','RP2','RP3','RP4'], axis=1,inplace=True)

#can drop all HU variables except HU2 (correlated wuth HU1) because it can tell us the probability that the donor is a 
#renter or not(if he/she is a renter means they are mobile/unstable (lower family loading??) but also with more bills to pay)
#keep HU5 to know if the neighbourhood is only a holiday one so only occassionally inhabited
donors.drop(['HU1','HU3','HU4'], axis=1,inplace=True)

#can drop all IC variables except IC3, IC5 because average, household and per capita are more meaningful than median (especially
#when they have the same distributions) and families
donors.drop(['IC1','IC2','IC4'], axis=1,inplace=True)

#can drop IC15-IC23, since we will focus on households rather than families' income
IC_list = donors.filter(regex='^IC',axis=1).columns.values[11:]
donors.drop(IC_list, axis=1,inplace=True)

#can drop all TPE variables as they are irrelevant to the donors' decision to donate (the mode of transportation of 
#the neighbourhood can hint to its income which we already have in IC variables)
TPE_list = donors.filter(regex='^TPE',axis=1).columns.values
donors.drop(TPE_list, axis=1,inplace=True)

#can drop all LFC variables as the neighbours' employment status will not affect the donors' behaviour
LFC_list = donors.filter(regex='^LFC',axis=1).columns.values
donors.drop(np.delete(LFC_list,9), axis=1,inplace=True)

#can drop all OCC, EIC since the type of employment of the neighbours 10 years ago will not affect the donation decision
OCC_list = donors.filter(regex='^OCC',axis=1).columns.values
EIC_list = donors.filter(regex='^EIC',axis=1).columns.values
donors.drop(np.concatenate([OCC_list,EIC_list]), axis=1,inplace=True)

#we will keep SEC1 & SEC2 because there is a culturalt distinction between people attending public vs private schools
#which can affect their moral values leading to donation decision
#can drop the rest of SEC since it does not provide enough information
donors.drop(['SEC3','SEC4','SEC5'], axis=1,inplace=True)

#can drop all AFCs and VCs except AFC1 and AFC4 because a neighbourhood with veterans & active military can be more 
#prone to donating to PVA 
donors.drop(['AFC2','AFC3','AFC5','AFC6'], axis=1,inplace=True)

#can drop all VCs because which war the veterans particpated in in a donor's neighbourhood will not affect his/her 
#willingness to donate...too specific
donors.drop(['VC1','VC2','VC3','VC4'], axis=1,inplace=True)

#can drop all HC except HC15 (solar energy are environmentally friendly...aka culture of giving back)
HC_list = donors.filter(regex='^HC',axis=1).columns.values
donors.drop(np.delete(HC_list,14), axis=1,inplace=True)

#can drop CARDPROM since redundant with NUMPROM (and NUMPROM is more informative since it includes all types of promotions)
#we don't want to lose the information about the donors who did not receive a card promotions
donors.drop('CARDPROM', axis=1,inplace=True)

#can drop CARDGIFT since redundant with NGIFTALL (and NGIFTALL is more informative since it includes alltypes of promotions)
donors.drop('CARDGIFT', axis=1,inplace=True)

#can drop the CHIL variables since AGE907 sums them up
donors.drop(['CHIL1','CHIL2','CHIL3'], axis=1,inplace=True)

#can drop HUR1 and HUR2 because they are irrelevant (no of rooms per household per neighbourhood is too specific)
donors.drop(['HUR1','HUR2'], axis=1,inplace=True)

#can drop ADI code (Approved Driving Instructors code) because the type of driving rules followed per region in the 
#US would not affect donor behaviour
donors.drop('ADI', axis=1,inplace=True)

#can drop MC variables due to lack of relevancy especially because 2005 is too far back
donors.drop(['MC1','MC2','MC3'], axis=1,inplace=True)

# drop incoherent variables
donors.drop(['EC1','MHUC1','MHUC2'], axis=1,inplace=True)

In [78]:
non_metric_features = ['STATE','RECPGVG','RECSWEEP','INCOME', 'GENDER', 'WEALTH2',
                       'DATASRCE', 'SOLP3', 'SOLIH', 'COLLECT1',
                       'VETERANS', 'BIBLE', 'CATLG', 'HOMEE', 'PETS', 'CDPLAY', 'STEREO',
                       'PCOWNERS', 'PHOTO', 'CRAFTS', 'FISHER', 'GARDENIN', 'BOATS',
                       'WALKER', 'KIDSTUFF', 'CARDS', 'PLATES','PEPSTRFL','MSA','DMA',
                       'RFA_2F', 'RFA_2A','urbanicity_level_neighbourhood', 'socio_econ_neighbourhood',
                       'MDMAUD_R','MDMAUD_F', 'MDMAUD_A']

In [79]:
# create a list with the metric features
all_features = list(donors.columns.values)
for non_metric_feature in non_metric_features:
    all_features.remove(non_metric_feature)
metric_features = all_features
#metric_features

**Missing Values**

In [80]:
# Percentage of missing values 
NA = donors.isna().mean()*100
NA[NA!=0]

INCOME                            22.387350
WEALTH2                           46.281376
MSA                                0.129606
DMA                                0.129606
AGE                               25.198730
urbanicity_level_neighbourhood     2.355597
socio_econ_neighbourhood           2.355597
dtype: float64

In [81]:
#donors['MSA'].value_counts().head(10) # predominant value - 0 => lets use the mode

In [82]:
#donors['DMA'].value_counts().head(10) # It doesnt seem to have a predominant value but givent that we dont know the meaning/importance the mode should be efficient

In [83]:
modes = donors[['MSA','DMA']].mode().loc[0]
donors.fillna(modes, inplace=True)

In [84]:
## Treat high cardinality for both variables
msa = donors['MSA'].value_counts().index[:3].tolist()
dma = donors['DMA'].value_counts().index[:3].tolist()
donors['MSA'] = np.where(~donors['MSA'].isin(msa), 'Other', donors['MSA'])
donors['DMA'] = np.where(~donors['DMA'].isin(dma), 'Other', donors['DMA'])

In [85]:
#donors.groupby('NUMCHLD')['AGE'].value_counts().unstack()

In [86]:
#n_child=[0,1,2,3,4,5,6,7]
#for n in n_child:
#    print(donors[donors['NUMCHLD']==n].groupby(['NUMCHLD'])['AGE'].value_counts()[0:3])

In [87]:
#replace_age1 = donors.groupby(['NUMCHLD'])['AGE'].apply(lambda x: x.value_counts().index[0])
#replace_age1

In [88]:
replace_age2 = donors.groupby(['NUMCHLD'])['AGE'].median()
#replace_age2

In [89]:
#donors['AGE'].mode()

In [90]:
# Opt for median - create new series with the dictionary association
AGE_CHILD = donors.NUMCHLD.replace(replace_age2)

In [91]:
donors['AGE'] = np.where(pd.isnull(donors.AGE), AGE_CHILD , donors.AGE)

In [92]:
# Creating new df copy to explore neighbordhood imputation
donors_neighbors = donors.copy()

# Seeing rows with NaNs
#nans_index = donors_neighbors.isna().any(axis=1)
#donors_neighbors[nans_index]

# KNNImputer - only works for numerical variables
imputer = KNNImputer(n_neighbors=4, weights="uniform")

# to fill the missing values of INCOME, WEALTH2 and DOMAIN (socio_econ_neighbourhood and urbanicity_level_neighbourhood)
related_variables = (['INCOME','WEALTH2','socio_econ_neighbourhood','urbanicity_level_neighbourhood', 'RFA_2A', 'MDMAUD_A', 'AVGGIFT','MAXRAMNT','MINRAMNT'] +
                     list(donors.filter(regex='^HVP',axis=1).columns.values) +
                     list(donors.filter(regex='^IC',axis=1).columns.values))

In [93]:
donors_neighbors[related_variables] = imputer.fit_transform(donors_neighbors[related_variables])

In [94]:
# drop WEALTH2 after helping filling the income values
donors_neighbors.drop('WEALTH2', axis=1, inplace=True)

In [95]:
# See rows with NaNs imputed
#donors_neighbors.loc[nans_index, related_variables]

In [96]:
filled_variables= ['INCOME','socio_econ_neighbourhood','urbanicity_level_neighbourhood']
for filled in filled_variables:
    donors_neighbors[filled] = donors_neighbors[filled].apply(lambda x: int(x+0.5))
    print(donors_neighbors[filled].unique())

[4 6 3 1 5 2 7]
[2 1 3 4]
[3 2 4 0 1]


In [97]:
# let's keep the imputation
donors = donors_neighbors.copy()

In [98]:
# use OneHotEncoder to encode the categorical features as dummy variable encoding
non_metric_encoding = ['STATE','MDMAUD_R','DATASRCE','GENDER','urbanicity_level_neighbourhood','MSA','DMA']
ohc = OneHotEncoder(sparse=False, drop="first")
ohc_feat = ohc.fit_transform(donors[non_metric_encoding])
ohc_feat_names = ohc.get_feature_names(['STATE','MDMAUD_R','DTSRCE','GENDER','urbanicity_level','MSA','DMA'])
ohc_donors = pd.DataFrame(ohc_feat, index=donors.index, columns=ohc_feat_names)  

donors = pd.concat([donors.drop(columns=non_metric_encoding), ohc_donors], axis=1)

**Final outlier removal**

In [99]:
# Pandas profiling
#ProfileReport(donors[metric_features], minimal=True)

In [100]:
donors.drop(['HVP3','HVP4','HVP5'],axis=1,inplace=True)

In [101]:
#donors[donors['POP90C4'] == 0]['POP90C5'].value_counts()
#donors['FM'] = donors['POP90C4'] + donors['POP90C5'] 
#donors[(donors['FM']==99)][['POP90C4','POP90C5']]
#donors.drop(['FM'],axis=1,inplace=True)

In [102]:
#len(donors_original[((donors_original['POP90C4'] == 0) & (donors_original['POP90C5'] == 0))])/len(donors_original) *100 # 0.8374

In [103]:
filter_to_drop = filter_to_drop | ((donors_original['POP90C4'] == 0) & (donors_original['POP90C5'] == 0))

In [104]:
donors = donors[((donors['POP90C4'] != 0) & (donors['POP90C5'] != 0))]

In [105]:
#len(donors_original[filter_to_drop])/len(donors_original) *100 # 3.765

In [106]:
donors.drop(['POP90C4'],axis=1,inplace=True)

In [107]:
# check 0's in common - 'POP901' and 'AGE904' 768 donors
#donors[donors['POP901'] == 0]['AGE904'].unique() # empty, 0's in common with previous variables

In [108]:
donors['area'] = donors['POP90C1'] + donors['POP90C2'] + donors['POP90C3']
#donors['area'].value_counts() #99,100,101,0

In [109]:
filter_to_drop = filter_to_drop | ((donors_original['POP90C1'] == 0) & (donors_original['POP90C2'] == 0) & (donors_original['POP90C3'] == 0))
#len(donors_original[filter_to_drop])/len(donors_original) *100 # 3.816

In [110]:
donors = donors[donors['area']!=0]
donors.drop(['area'],axis=1,inplace=True)

In [111]:
# Transform this variables into dummies
donors.drop(['POP90C2'], axis=1, inplace=True)
donors['POP90C1'] = np.where(donors['POP90C1'] < 50, 0, 1)
donors['POP90C3'] = np.where(donors['POP90C3'] < 50, 0, 1)

In [112]:
#AGEC_features = list(donors.filter(regex='^AGEC',axis=1).columns.values[0:4])
#AGEC_features.append(donors.filter(regex='^AGEC',axis=1).columns.values[6])
#for age in AGEC_features:
#    print(donors[donors[age]>30][age].value_counts())

In [113]:
filter_age = ((donors['AGEC1']>66) |
              (donors['AGEC2']>56) |
              (donors['AGEC3']>48) |
              (donors['AGEC4']>34) |
              (donors['AGEC7']>61))
donors_2 = donors[~filter_age]

In [114]:
#print('Percentage of data kept after removing outliers:', np.round(donors_2.shape[0] / donors.shape[0], 4)) #0.9974

In [115]:
filter_age_original = ((donors_original['AGEC1']>66) |
              (donors_original['AGEC2']>56) |
              (donors_original['AGEC3']>48) |
              (donors_original['AGEC4']>34) |
              (donors_original['AGEC7']>61))
#donors_2_original = donors_original[~filter_age_original]
#print('Percentage of data kept after removing outliers:', np.round(donors_2_original.shape[0] / donors_original.shape[0], 4)) #0.997

In [116]:
filter_to_drop = filter_to_drop | (filter_age_original)

In [117]:
#len(donors_original[filter_to_drop])/len(donors_original) * 100 #4.0697

In [118]:
# keep this alteration
donors = donors_2.copy()

In [119]:
remove_features = ['NUMCHLD','HVP3','HVP4','HVP5','POP90C1','POP90C2','POP90C3','POP90C4']
for f in remove_features:
    metric_features.remove(f)

**Multivariate Outliers**

In [120]:
##### Extended Isolation Forest

# matrix with values
X = donors[metric_features].values

clf = IsolationForest(n_estimators=300, random_state=42, n_jobs=-1,
                     behaviour='new',
                     max_samples = 'auto', contamination='auto',
                     max_features=1.0,
                     bootstrap=True)
clf.fit(X)
isolation_forest = clf.predict(X)

C:\Users\betty\anaconda3\lib\site-packages\sklearn\ensemble\_iforest.py:252: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  warn(


In [121]:
isolation_forest_outliers = [True if x == -1 else False for x in isolation_forest]
#sum(isolation_forest_outliers*1) # 1087 outliers

1087

In [122]:
donors2 = donors[np.logical_not(isolation_forest_outliers)]

In [123]:
donors_original = pd.concat([donors_original,pd.Series(isolation_forest_outliers,index=donors.index,name = 'multiOut')],axis=1)

In [124]:
filter_to_drop = filter_to_drop | (donors_original['multiOut'] == True)
#len(donors_original[filter_to_drop])/len(donors_original) *100 # 5.208988

5.208988387204964

In [125]:
# keep data without multi outliers
donors = donors2.copy()
donors_original.drop(['multiOut'], axis = 1, inplace = True)

In [127]:
# Percentage of data kept 
#len(donors)/len(donors_original) * 100 # 94.787867

94.78786735421122

In [128]:
# columns difference (with dummies already ...)
#donors.shape[1]/donors_original.shape[1] * 100 # 32.84

32.8421052631579

In [129]:
#donors.to_csv(r"C:\Users\betty\OneDrive\Documentos\donors_preprocessed.csv")